In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import re


driver = webdriver.Chrome()
urlP = "WEB URL"   #  <---------------
driver.get(urlP)



#scroll SETTİNGS 


scroll_pause = 2
scroll_step = 1000
scroll_limit = 1000     
check_interval = 15     

last_height = driver.execute_script("return document.body.scrollHeight")
unchanged_count = 0     

for i in range(1, scroll_limit + 1):
    driver.execute_script(f"window.scrollBy(0, {scroll_step});")
    time.sleep(scroll_pause)

    if i % check_interval == 0:  
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            unchanged_count += 1
            print(f"⚠️ {unchanged_count}. kez yükseklik değişmedi (step {i})")
        else:
            unchanged_count = 0  
            last_height = new_height

        if unchanged_count >= 2:  
            print(f"✅ Sayfa sonuna ulaşıldı, scroll durduruldu. (step {i})")
            break





soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

data_value= []

comment_Text = soup.find_all("div", class_="comment")    #   <--------------------

for comment in comment_Text:
    comment_div = comment.find("div", class_="comment-text")#   <--------------------
    
    comment_p = comment_div.find("p") if comment_div else None
    
    comment_text = comment_p.text.strip() if comment_p else "(command not found)"    #   <--------------------

    star_blocks = comment.find_all("div", class_="star-w")    #   <--------------------
    puan = 0

    for star in star_blocks:
        full = star.find("div", class_="full")   #   <--------------------
        if full:
            style = full.get("style", "")
            match = re.search(r'width:\s*(\d+)', style)
            if match:
                width = int(match.group(1))
                if width >= 75:
                    puan += 1
                elif 25 <= width < 75:
                    puan += 0.5

    data_value.append({
        "commnet": comment_text,
        "puan": puan
    })


df = pd.DataFrame(data_value)
print(df)


In [ ]:

from datetime import datetime


date-time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


file_name = f"Comments_{date-time}.csv"


df.to_csv(dfile_name, index=False, encoding="utf-8-sig")
print(f"Kaydedildi: {file_name}")
